In [1]:
import pandas as pd
from linkml_owl.owl_dumper import OWLDumper
from linkml_runtime import SchemaView
from linkml_runtime.utils.introspection import package_schemaview

from loinc_owl.part_schema import ComponentClass, LoincCodeOntology
from linkml_runtime.dumpers import json_dumper


In [11]:
parts = pd.read_csv("data/PartFile/Part.csv")
part_supp = pd.read_csv("data/PartFile/LoincPartLink_Supplementary.csv")
# part_prim = pd.read_csv("data/PartFile/LoincPartLink_Primary.csv")
# part_map = pd.read_csv("data/PartFile/PartRelatedCodeMapping.csv)
ma_hierarchy = pd.read_csv("data/MultiAxialHierarchy/MultiAxialHierarchy.csv")

In [3]:
from collections import defaultdict
parent_lookup = defaultdict(list)
for ma_code in ma_hierarchy.itertuples():
    parent_lookup[ma_code.CODE].append(ma_code.IMMEDIATE_PARENT)

In [37]:
len(set(ma_hierarchy['CODE'].tolist()))

171191

In [12]:
part_supp

,LoincNumber,LongCommonName,PartNumber,PartName,PartCodeSystem,PartTypeName,LinkTypeName,Property
0,100002-5,Specimen care is maintained,LP430721-3,Specimen care is maintained,http://loinc.org,COMPONENT,DetailedModel,http://loinc.org/property/analyte
1,100002-5,Specimen care is maintained,LP6813-2,Find,http://loinc.org,PROPERTY,DetailedModel,http://loinc.org/property/PROPERTY
2,100002-5,Specimen care is maintained,LP6960-1,Pt,http://loinc.org,TIME,DetailedModel,http://loinc.org/property/time-core
3,100002-5,Specimen care is maintained,LP7751-3,Ord,http://loinc.org,SCALE,DetailedModel,http://loinc.org/property/SCALE_TYP
4,100002-5,Specimen care is maintained,LP6985-8,Patient,http://loinc.org,SUPER SYSTEM,DetailedModel,http://loinc.org/property/super-system
...,...,...,...,...,...,...,...,...
1224203,99998-7,Neurological status is maintained at or improv...,LP430720-5,Neurological status is maintained at or improv...,http://loinc.org,COMPONENT,SyntaxEnhancement,http://loinc.org/property/analyte-core
1224204,99998-7,Neurological status is maintained at or improv...,LP73412-6,Status,http://loinc.org,COMPONENT,Search,http://loinc.org/property/search
1224205,99998-7,Neurological status is maintained at or improv...,LP89782-4,Neurological,http://loinc.org,COMPONENT,Search,http://loinc.org/property/search
1224206,99998-7,Neurological status is maintained at or improv...,LP29696-9,Survey instruments,http://loinc.org,CLASS,Metadata,http://loinc.org/property/category


In [13]:
code_name_lookup = {x.PartNumber: x.PartName for x in part_supp.itertuples()}

PartCode PartName PartType ParentCode

In [22]:
code_name_lookup

{'LP430721-3': 'Specimen care is maintained',
 'LP6813-2': 'Find',
 'LP6960-1': 'Pt',
 'LP7751-3': 'Ord',
 'LP6985-8': 'Patient',
 'LP29696-9': 'Survey instruments',
 'LP430695-9': 'SURVEY.PNDS',
 'LP430722-1': 'Team communication is maintained throughout care',
 'LP267494-5': 'Communication',
 'LP430723-9': 'Demonstrates knowledge of the expected psychosocial responses to the procedure',
 'LP73191-6': 'Procedure',
 'LP430724-7': 'Demonstrates knowledge of nutritional management related to the procedure',
 'LP89783-2': 'Nutritional',
 'LP430725-4': 'Demonstrates knowledge of medication management',
 'LP116700-8': 'Medication',
 'LP430726-2': 'Demonstrates knowledge of pain management',
 'LP75331-6': 'Pain',
 'LP89789-9': 'Pain management',
 'LP31088-5': 'R wave duration.lead AVR',
 'LP6879-3': 'Time',
 'LP7289-4': 'Heart',
 'LP7753-9': 'Qn',
 'LP6244-0': 'EKG',
 'LP14259-3': 'Lead',
 'LP14744-4': 'Duration',
 'LP31112-3': "R' wave duration.lead AVR",
 'LP190563-9': 'Cardiology',
 'LP29

In [19]:
cnl_keys = code_name_lookup.keys()

In [14]:
component_system_hierarchy = pd.read_excel('data/CHEM_HIERARCHY_LPL_DATA.xlsx', sheet_name="Hierarchy").astype(str)
lpl_data =  pd.read_excel('data/CHEM_HIERARCHY_LPL_DATA.xlsx', sheet_name="LPL").astype(str)

In [15]:
lpl_data = lpl_data[['PART_NUM', 'PART', 'NAME']].drop_duplicates()

In [24]:
def get_parent_code(code):
    parent_code_list = parent_lookup[code]
    if not len(parent_code_list):
        parent_code_list.append("000001")
    return parent_code_list

def get_code_name(code):
    try:
        return code_name_lookup[code]
    except:
        return "Unnamed"
def loincify(id):
    return f"loinc:{id}"

In [30]:
lpl_data

,PART_NUM,PART,NAME
0,LP100008-4,Oxygen capacity,COMPONENT
2,LP100018-3,Arylsulfatase C,COMPONENT
4,LP100176-9,2-Methylacetoacetate,COMPONENT
6,LP100184-3,Starch,COMPONENT
7,LP100320-3,Clinical biochemist review,COMPONENT
...,...,...,...
26884,LP99929-9,Gamma tocopherol,COMPONENT
26887,LP99931-5,Transcobalamin I+Transcobalamin III,COMPONENT
26888,LP99932-3,Uridine,COMPONENT
26890,LP99934-9,Alanine glyoxylate aminotransferase,COMPONENT


In [33]:
# sv = package_schemaview('loinc_owl.part_schema')
sv = SchemaView('model/schema/part_schema.yaml')
od = OWLDumper()

component_classes = []
for index, lpl in enumerate(lpl_data.itertuples()):
    parents_list = get_parent_code(lpl.PART_NUM)
    parents_curie_list = [loincify(x) for x in parents_list]
    for parent in parents_list:
        parent_curie = loincify(parent)
        parent_name = get_code_name(parent)
        parent_class = ComponentClass(id=loincify(parent), subClassOf="loinc:000000001", label=parent_name)
        component_classes.append(parent_class)
    print(lpl.PART_NUM)
    component_class = ComponentClass(id=loincify(lpl.PART_NUM), subClassOf=parents_curie_list, label=lpl.PART)
    component_classes.append(component_class)
with open("data/output/component_classes.owl", 'w') as ccl_owl:
    ccl_owl.write(od.dumps(component_classes, schema=sv.schema))
loinc_ontology_class = LoincCodeOntology(component_class_set=component_classes)




# #
# # with open("../data/output/component_classes.json", 'w') as ccl_json:
# #     ccl_json.write(json_dumper.dumps(loinc_ontology_class))
# #     # joined = ',\n'.join([json_dumper.dumps(x) for x in component_classes])
# #     # ccl_json.write(f"[\n{joined}\n]")





LP100008-4
LP100018-3
LP100176-9
LP100184-3
LP100320-3
LP100774-1
LP100837-6
LP101306-1
LP101460-6
LP101481-2
LP101811-0
LP104997-4
LP104998-2
LP105135-0
LP111163-4
LP111164-2
LP111174-1
LP111176-6
LP111178-2
LP111180-8
LP111182-4
LP111251-7
LP111253-3
LP111254-1
LP111255-8
LP111484-4
LP113712-6
LP113909-8
LP113910-6
LP113911-4
LP113916-3
LP113917-1
LP113963-5
LP128556-0
LP133415-2
LP133642-1
LP133649-6
LP133650-4
LP133973-0
LP134393-0
LP134394-8
LP14012-6
LP14013-4
LP14014-2
LP14015-9
LP14016-7
LP14017-5
LP14018-3
LP14019-1
LP14020-9
LP14021-7
LP140221-5
LP14022-5
LP14023-3
LP14024-1
LP14025-8
LP14026-6
LP14027-4
LP14028-2
LP14029-0
LP14031-6
LP14032-4
LP14033-2
LP14034-0
LP14035-7
LP14040-7
LP14041-5
LP14045-6
LP14071-2
LP14161-1
LP14270-0
LP14283-3
LP14288-2
LP14296-5
LP14299-9
LP14310-4
LP14329-4
LP14331-0
LP14333-6
LP14334-4
LP14335-1
LP14336-9
LP14337-7
LP14338-5
LP14339-3
LP14340-1
LP14341-9
LP14343-5
LP14345-0
LP14355-9
LP14356-7
LP14369-0
LP14372-4
LP14377-3
LP14438-3
LP14449-

LP17145-1
LP17170-9
LP17171-7
LP17184-0
LP17185-7
LP17186-5
LP17187-3
LP171883-4
LP17229-3
LP17233-5
LP17234-3
LP17248-3
LP172597-9
LP17263-2
LP172681-1
LP172715-7
LP172734-8
LP172773-6
LP172793-4
LP172794-2
LP172797-5
LP172798-3
LP172845-2
LP173633-1
LP173634-9
LP173639-8
LP173640-6
LP173648-9
LP173656-2
LP173658-8
LP173710-7
LP173714-9
LP173716-4
LP173717-2
LP173718-0
LP173719-8
LP173720-6
LP173721-4
LP173725-5
LP173998-8
LP173999-6
LP174001-0
LP174002-8
LP174005-1
LP174010-1
LP174011-9
LP174016-8
LP174017-6
LP174026-7
LP174038-2
LP174039-0
LP174051-5
LP174062-2
LP174065-5
LP174073-9
LP174075-4
LP174076-2
LP174077-0
LP174078-8
LP174084-6
LP174089-5
LP174092-9
LP174093-7
LP174095-2
LP174096-0
LP174098-6
LP174099-4
LP174557-1
LP174558-9
LP174560-5
LP174561-3
LP174562-1
LP175628-9
LP175759-2
LP17650-0
LP17770-6
LP17841-5
LP17852-2
LP17856-3
LP17876-1
LP17877-9
LP17878-7
LP17893-6
LP17895-1
LP17921-5
LP17945-4
LP17957-9
LP18007-2
LP18021-3
LP18022-1
LP18032-0
LP18033-8
LP18034-6
LP18035-

LP285075-0
LP285076-8
LP285077-6
LP285078-4
LP285079-2
LP285080-0
LP285081-8
LP285082-6
LP285083-4
LP285084-2
LP285085-9
LP285086-7
LP285087-5
LP285090-9
LP285091-7
LP285092-5
LP285093-3
LP285094-1
LP285098-2
LP285099-0
LP285100-6
LP285101-4
LP285102-2
LP285163-4
LP285182-4
LP285183-2
LP285184-0
LP285185-7
LP285186-5
LP285187-3
LP285189-9
LP285190-7
LP285191-5
LP285192-3
LP285193-1
LP285194-9
LP285195-6
LP285196-4
LP285197-2
LP285198-0
LP285199-8
LP285200-4
LP28520-2
LP285212-9
LP285215-2
LP285221-0
LP285222-8
LP285223-6
LP285224-4
LP285225-1
LP285226-9
LP285227-7
LP285228-5
LP285229-3
LP285230-1
LP285231-9
LP285232-7
LP285245-9
LP285246-7
LP28532-7
LP28537-6
LP28539-2
LP28550-9
LP28551-7
LP28558-2
LP28565-7
LP285734-2
LP285737-5
LP285738-3
LP285739-1
LP285740-9
LP285741-7
LP285742-5
LP285743-3
LP285744-1
LP285745-8
LP285746-6
LP285747-4
LP285748-2
LP285749-0
LP285750-8
LP285751-6
LP285752-4
LP285753-2
LP285754-0
LP285755-7
LP285756-5
LP285760-7
LP285767-2
LP285768-0
LP285769-8
LP28577

LP35604-5
LP35625-0
LP35639-1
LP35640-9
LP35677-1
LP35723-3
LP35730-8
LP35745-6
LP35748-0
LP35751-4
LP35752-2
LP35767-0
LP35889-2
LP35893-4
LP35919-7
LP35921-3
LP36035-1
LP36036-9
LP36040-1
LP36161-5
LP36186-2
LP36361-1
LP36420-5
LP36421-3
LP36425-4
LP36470-0
LP36471-8
LP36472-6
LP36474-2
LP36475-9
LP36498-1
LP36574-9
LP36594-7
LP36595-4
LP36603-6
LP36604-4
LP36605-1
LP36606-9
LP36607-7
LP36663-0
LP36679-6
LP36680-4
LP36718-2
LP36719-0
LP36720-8
LP36726-5
LP36731-5
LP36732-3
LP36734-9
LP36737-2
LP36760-4
LP36785-1
LP36786-9
LP36792-7
LP36802-4
LP36805-7
LP36806-5
LP36807-3
LP36808-1
LP36809-9
LP36872-7
LP36886-7
LP37834-6
LP38608-3
LP38609-1
LP38610-9
LP38618-2
LP38619-0
LP38620-8
LP38712-3
LP38713-1
LP38714-9
LP39043-2
LP39811-2
LP40002-5
LP40007-4
LP40009-0
LP40010-8
LP40225-2
LP40334-2
LP40352-4
LP40488-6
LP40499-3
LP410748-0
LP410749-8
LP411027-8
LP411029-4
LP411030-2
LP411031-0
LP411032-8
LP411033-6
LP411034-4
LP411035-1
LP411037-7
LP411038-5
LP411039-3
LP411042-7
LP411049-2
LP411

ValueError: loinc:000001: duplicate key

In [35]:
print(len(list(lpl_data.itertuples())))

3430
